In [5]:
!pip install rdflib
import types
import typing
import os
import re

import pandas

import rdflib
from rdflib import RDF, OWL, XSD

## Parsing and printing ontology:

In [6]:
g = rdflib.Graph()

g.parse("NetworkOntology.owl", format="xml")

print("graph has {} statements.".format(len(g)))
print(g.serialize(format="turtle").decode("utf-8"))


graph has 225 statements.
@prefix : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology> a owl:Ontology .

<https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11a/h/j/n/ac/ax_frequencyRange> a owl:NamedIndividual,
        :FrequencyRange ;
    :frequencyRangeHigherBound 5.895e+03 ;
    :frequencyRangeLowerBound 4.91e+03 .

<https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11ad/ay__frequencyRange> a owl:NamedIndividual,
        :FrequencyRange ;
    :frequencyRangeHigherBound 7.02e+04 ;
    :frequencyRangeLowerBound 5.724e+04 .

:802.11ah_frequencyRange a owl:NamedIndividual,
        :FrequencyRange ;
    :frequencyRangeHigherBound 9.28e+02 ;
    :frequencyRangeLowerBou

### Print all classes in ontology

In [7]:
classes = g.triples((None, RDF.type, OWL.Class))
for s,v,o in classes:
  print(s, v, o)


N19c4e62f979144cbac045de461d7e83e http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#NetworkDevice http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#ProtocolDataType http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#FrequencyRange http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
Na2d2743917ab43648ab4942d804ee7cd http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
Na68016c8c09e45e8858de2ffb0597249 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#WirelessPhysicalProtocol http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#C

### Print Object Properties:

In [8]:
objectProps = g.triples((None, RDF.type, OWL.ObjectProperty))
for s,v,o in objectProps:
  print(s, v, o)


https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#hasFrequencyRange http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#deviceHasFrequencyRange http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#hasPDU http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#deviceForProtocol http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#deviceForOsiLayer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#ObjectProperty
https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#belongsToLayer http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/200

## Populate data

### Get Classes and required data properties:

In [9]:
namespace = rdflib.Namespace('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#')

wifiNetworkModuleClass = namespace.WiFiNetworkModule
bluetoothNetworkModuleClass = namespace.BluetoothModule

deviceMinTemperature = namespace.deviceMinTemperature
deviceMaxTemperature = namespace.deviceMaxTemperature
deviceDataRate = namespace.deviceDataRate
deviceManufacturer = namespace.deviceManufacturer
deviceForProtocol = namespace.deviceForProtocol


### Defining helpers for filling RDF graph from pandas 

In [10]:

RowToSubject = typing.Callable[[pandas.Series], typing.Tuple[rdflib.URIRef, rdflib.URIRef, rdflib.URIRef]]
ColToPredicate = typing.Callable[[pandas.Series], rdflib.URIRef]
CellToObject = typing.Callable[[str], rdflib.URIRef]

def col_to_predicate_from_dict(col_to_predicate_dict: typing.Dict[str, rdflib.URIRef]) -> ColToPredicate:
  return lambda col: col_to_predicate_dict[col.name] if col.name in col_to_predicate_dict else None

def cell_to_literal(datatype: rdflib.URIRef):
  return lambda v: rdflib.Literal(v, datatype=datatype) if v is not None and len(str(v)) != 0 else None

class RdfGraphFiller:
  def __init__(self, row_to_subject: RowToSubject, col_to_predicate: ColToPredicate, cell_to_object: typing.Dict[str, CellToObject]):
    self.row_to_subject = row_to_subject
    self.col_to_predicate = col_to_predicate
    self.cell_to_object = cell_to_object
  
  def fill_from_dataframe(self, graph: rdflib.Graph, df: pandas.DataFrame):
    for idx, r in df.iterrows():
      subject = self.__get_subject(graph, r)
      for col_name, cell_value in r.items():
        col = df[col_name]
        predicate = self.col_to_predicate(col)
        objects = self.__get_objects(g, col, cell_value)
        if subject is not None and predicate is not None:
          for obj in objects:
            g.add((subject, predicate, obj))
  

  def __get_subject(self, graph: rdflib.Graph, r: pandas.Series):
    subject = self.row_to_subject(r)
    self.__add_to_graph_if_required(graph, subject)
    return subject[0]

  def __get_objects(self, graph: rdflib.Graph, col: pandas.Series, cell_value):
    objs = self.cell_to_object[col.name](cell_value) if col.name in self.cell_to_object else None
    if type(objs) is not list:
      objs = [objs]
    objs_converted = []
    for obj in objs:
      if self.__add_to_graph_if_required(graph, obj):
        objs_converted.append(obj[0])
      elif obj is not None:
        objs_converted.append(obj) 
    return objs_converted

  def __add_to_graph_if_required(self, graph: rdflib.Graph, v):
    if type(v) is tuple and len(v) == 3:
      g.add(v)
      return True
    return False


### Add 802.11 protocols

In [11]:
def char_range(c1, c2):
    """Generates the characters from `c1` to `c2`, inclusive."""
    for c in range(ord(c1), ord(c2)+1):
        yield chr(c)

OSIProtocol802_11 = rdflib.URIRef(namespace + '802.11')
protocols_postfixes = [c for c in char_range('a', 'y')]
protocols_postfixes += ['ac', 'ad', 'ax', 'ay', 'az', 'be']

protocols_802_11 = set([rdflib.URIRef(namespace + '802.11' + postfix) for postfix in protocols_postfixes])
frequency_ranges = list(g.triples((None, RDF.type, namespace.FrequencyRange)))

for protocol in protocols_802_11:
  g.add((protocol, RDF.type, OSIProtocol802_11))
  proto_letter = protocol[protocol.index('802.11') + len('802.11'):]
  freq_range_regex = r'.*802\.1(1' + proto_letter + '(/|_)|1.*/' + proto_letter +'(/|_))'
  matched_freq_ranges = list(filter(lambda r: re.match(freq_range_regex, r[0]) is not None, frequency_ranges))
  for (r, _, _) in matched_freq_ranges:
    g.add((protocol, namespace.hasFrequencyRange, r))

### Filling WiFiNetwrokModules properties

In [12]:
manufacturer_key = 'Mfr.'
max_temp_key = 'Maximum Operating Temperature'
min_temp_key = 'Minimum Operating Temperature'
data_rate_key = 'Data Rate'
wifi_protocol_key = 'Protocol Supported'

non_decimal = re.compile(r'[^\d.-]+')

wifi_row_to_subject: RowToSubject = lambda r: (rdflib.URIRef(namespace + r['Mouser Part Number']), RDF.type, wifiNetworkModuleClass)
wifi_col_to_predicate: ColToPredicate = col_to_predicate_from_dict(
    {
      manufacturer_key: deviceManufacturer,
      max_temp_key: deviceMaxTemperature,
      min_temp_key: deviceMinTemperature,
      data_rate_key: deviceDataRate,
      wifi_protocol_key: deviceForProtocol
    }
)

def str_to_wifi_protocol(protocol):
  protocol = str(protocol).replace(' ', '')
  if not protocol.startswith('802.11'):
    return None
  
  protocol_sections = protocol.split('/')
  protocols = [protocol_sections[0]]
  for protocol_postfix in protocol_sections[1:]:
    protocols.append("802.11" + protocol_postfix)
  
  protocols = [rdflib.URIRef(namespace + protocol_name) for protocol_name in protocols]
  protocols = filter(lambda p: p in protocols_802_11, protocols)
  return [protocol for protocol in protocols]
  


def temperature_str_to_literal(cell_value):
  digits = non_decimal.sub('', str(cell_value))
  if len(digits) == 0:
    return None
  else:
    return rdflib.Literal(digits, datatype=XSD.double)

wifi_col_and_cel_to_object: typing.Dict[str, CellToObject] = {
  manufacturer_key: cell_to_literal(XSD.string),
  max_temp_key: temperature_str_to_literal,
  min_temp_key: temperature_str_to_literal,
  data_rate_key: cell_to_literal(XSD.string),
  wifi_protocol_key: str_to_wifi_protocol
}
wifi_modules = pandas.read_csv("wifi-data.csv").fillna('')

converter = RdfGraphFiller(wifi_row_to_subject, wifi_col_to_predicate, wifi_col_and_cel_to_object)
converter.fill_from_dataframe(g, wifi_modules)

### Filling BluetoothNetworkDevice module

In [13]:
manufacturer_key = 'Mfr.'
max_temp_key = 'Maximum Operating Temperature'
min_temp_key = 'Minimum Operating Temperature'
data_rate_key = 'Data Rate'
bluetooth_protocol_key = 'Protocol'

non_decimal = re.compile(r'[^\d.-]+')

def str_to_bluetooth_protocol(protocol):
  protocol = str(protocol).replace(' ', '')
  if not protocol.startswith('Bluetooth'):
    return None
  
  m = re.search(r'Bluetooth\d\.\d', protocol)
  if m is None:
    return None
  protocol_name = m.group(0)
  return [rdflib.URIRef(namespace + protocol_name)]
  
  #protocols = [rdflib.URIRef(namespace + protocol_name) for protocol_name in protocols]
  #return [protocol for protocol in protocols]
  

bluetooth_row_to_subject: RowToSubject = lambda r: (rdflib.URIRef(namespace + r['Mouser Part Number']), RDF.type, bluetoothNetworkModuleClass)
bluetooth_col_to_predicate: ColToPredicate = col_to_predicate_from_dict(
    {
      manufacturer_key: deviceManufacturer,
      max_temp_key: deviceMaxTemperature,
      min_temp_key: deviceMinTemperature,
      data_rate_key: deviceDataRate,
      bluetooth_protocol_key: deviceForProtocol
    }
)

bluetooth_col_and_cel_to_object: typing.Dict[str, CellToObject] = {
  manufacturer_key: cell_to_literal(XSD.string),
  max_temp_key: temperature_str_to_literal,
  min_temp_key: temperature_str_to_literal,
  data_rate_key: cell_to_literal(XSD.string),
  bluetooth_protocol_key: str_to_bluetooth_protocol,
}
bluetooth_modules = pandas.read_csv("bluetooth-data.csv").fillna('')

converter = RdfGraphFiller(bluetooth_row_to_subject, bluetooth_col_to_predicate, bluetooth_col_and_cel_to_object)
converter.fill_from_dataframe(g, bluetooth_modules)

### Saving ontology


In [14]:
g.serialize("OutputOntology.owl")

## Generate Documentation

In [15]:
!pip install pylode

     |████████████████████████████████| 143kB 6.8MB/s 
     |████████████████████████████████| 61kB 4.1MB/s 
  Created wheel for rdflib-jsonld: filename=rdflib_jsonld-0.5.0-py2.py3-none-any.whl size=15348 sha256=0305c66e3fcfbecdedaebaf2123c9cfe295364b18003026d21587ededafb31ac
  Stored in directory: /root/.cache/pip/wheels/8d/e4/7f/9ebcb3e400c694e645c3adba40ef3e9bda78384ac3b9b0d13d
Successfully built rdflib-jsonld


In [16]:
import pylode

html = pylode.MakeDocco(
    input_data_file="OutputOntology.owl",
    outputformat="html"
).document()

file = open('doc.html', 'w')
file.write(html)
file.close()

## Generaiting VoID

In [17]:
from rdflib.void import generateVoID

void_graph = generateVoID(g)[0]
void_graph.serialize('void.rdf')
print(void_graph.serialize(format="turtle").decode("utf-8"))

@prefix ns1: <http://rdfs.org/ns/void#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/Dataset> a ns1:Dataset ;
    ns1:classPartition <http://example.org/Dataset_class0>,
        <http://example.org/Dataset_class1>,
        <http://example.org/Dataset_class10>,
        <http://example.org/Dataset_class11>,
        <http://example.org/Dataset_class12>,
        <http://example.org/Dataset_class2>,
        <http://example.org/Dataset_class3>,
        <http://example.org/Dataset_class4>,
        <http://example.org/Dataset_class5>,
        <http://example.org/Dataset_class6>,
        <http://example.org/Dataset_class7>,
        <http://example.org/Dataset_class8>,
        <http://example.org/Dataset_class9> ;
    ns1:classes 13 ;
    ns1:distinctObjects 208 ;
    ns1:distinctSubjects 465 ;
    ns1:properties 21 ;
    ns1:propertyPartition <http://

## Reasoning

In [18]:
!apt-get install -y openjdk8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = 'usr/lib/jvm/java8-openjdk-amd64'
!java -version
!pip install owlready2

E: Unable to locate package openjdk8-jdk-headless
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
     |████████████████████████████████| 23.7MB 53.6MB/s 
  Created wheel for owlready2: filename=Owlready2-0.31-cp37-cp37m-linux_x86_64.whl size=20409711 sha256=4e74e55da5f8a65ce55fa400aae2b600a626f90a66428484bb331fbd7d653086
  Stored in directory: /root/.cache/pip/wheels/9d/17/e4/54fb8381c51746a0d686b06ef3e3f607789defd8e1580bcb1b
Successfully built owlready2


In [19]:
from owlready2 import *
import owlready2

#ontology = get_ontology('OutputOntology.owl').load()

#with ontology:
#  sync_reasoner()
#  ontology.save('network-ontology-inferred.owl', format="rdfxml")

#SPARQL queries

### Open Inferred OWL

In [20]:
g = g.parse("network-ontology-inferred.owl")

### Which protocols belongs to OSI layer #1?

In [21]:
qres = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#>
  PREFIX inferred: <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology-inferred#>

  SELECT ?protocol ?layer ?layerNumber
	  WHERE  { 
		  ?protocol rdf:type :OSIProtocol.
		  ?protocol :belongsToLayer ?layer.
		  ?layer :hasLayerNumber ?layerNumber.
		  FILTER(?layerNumber = 1)
	  }
""")

for row in qres:
  print(row)

(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#Bluetooth4.0'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#physical'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11p'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#physical'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#Bluetooth1.0'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#physical'), rdflib.term.Literal('1', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11g'), rdflib.term.URIRef('https://ifmo.ru/etsile

### Which devices support 802.11n protocol?

In [22]:
qres = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#>
  PREFIX inferred: <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology-inferred#>

  
  SELECT ?device ?protocol
	  WHERE { 
		  ?protocol rdf:type :OSIProtocol.
		  ?device :deviceForProtocol ?protocol.
		  FILTER(?protocol = :802.11n)
	  }
""")

for row in qres:
  print(row)

(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#713-317060362'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11n'))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#634-WF111-E-V1'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11n'))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#924-EAR00323'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11n'))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#356-ESP32WROOM32D8MB'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11n'))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#963-WYSAGVDXG'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11n'))
(rdflib.term.URIRef('https://ifmo.ru/e

### Which devices can work on temperature +100 C

In [23]:
qres = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#>
  PREFIX inferred: <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology-inferred#>

  
  SELECT ?device ?minTemperature ?maxTemperature
	  WHERE { 
		  ?device rdf:type :NetworkDevice.
		  ?device :deviceMaxTemperature ?maxTemperature.
		  ?device :deviceMinTemperature ?minTemperature.
		  FILTER(?maxTemperature >= 100 && ?minTemperature <= 100)
	  }
""")

for row in qres:
  print(row)



(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#308-BM833E'), rdflib.term.Literal('-40.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('105.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#634-BGM220PC22HNA2R'), rdflib.term.Literal('-40.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('105.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#634-WF200SD'), rdflib.term.Literal('-40.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('105.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#377-NINA

### Which devices can work with temperature -30 C

In [24]:
qres = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#>
  PREFIX inferred: <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology-inferred#>

  
  SELECT ?device ?minTemperature ?maxTemperature
	  WHERE { 
		  ?device rdf:type :NetworkDevice.
		  ?device :deviceMaxTemperature ?maxTemperature.
		  ?device :deviceMinTemperature ?minTemperature.
		  FILTER(?maxTemperature >= -30 && ?minTemperature <= -30)
	  }
""")

for row in qres:
  print(row)


(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#308-BM833E'), rdflib.term.Literal('-40.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('105.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#634-BGM220PC22HNA2R'), rdflib.term.Literal('-40.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('105.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#364-ACN52832'), rdflib.term.Literal('-40.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('85.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#356-ESP3

### Which physical devices can work in frequency 5000MHz

In [25]:
qres = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#>
  PREFIX inferred: <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology-inferred#>

  
  SELECT ?device ?protocol ?freqLow ?freqHigh 
	  WHERE { 
		  ?device rdf:type :NetworkDevice.
		  ?device :deviceForOsiLayer :physical.
      ?device :deviceForProtocol ?protocol.
      ?protocol :hasFrequencyRange ?freqRange.
      ?freqRange :frequencyRangeLowerBound ?freqLow.
      ?freqRange :frequencyRangeHigherBound ?freqHigh.
      FILTER (?freqHigh >= 5000 && ?freqLow <= 5000)
	  }
""")

for row in qres:
  print(row)


(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#607-AX201.NGWG'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11ax'), rdflib.term.Literal('4910.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('5895.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#607-AX201NGWGW'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11a'), rdflib.term.Literal('4910.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), rdflib.term.Literal('5895.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')))
(rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#742-KIT-WIFI-UDOOX86'), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#802.11ac'), rdfl

### Which Manufacturers produce devices for Bluetooth protocols

In [26]:
qres = g.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX : <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#>
  PREFIX inferred: <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology-inferred#>

  
  SELECT ?manufacturer ?protocol
	  WHERE { 
		  ?device rdf:type :NetworkDevice.
      ?device :deviceForProtocol ?protocol.
      ?device :deviceManufacturer ?manufacturer.
      FILTER (regex(str(?protocol), "Bluetooth", "i"))
	  }
""")

for row in qres:
  print(row)


(rdflib.term.Literal('Fanstel', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#Bluetooth5.1'))
(rdflib.term.Literal('Microchip Technology', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#Bluetooth4.2'))
(rdflib.term.Literal('Silicon Labs', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#Bluetooth5.2'))
(rdflib.term.Literal('aconno', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#Bluetooth4.2'))
(rdflib.term.Literal('Cypress Semiconductor', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')), rdflib.term.URIRef('https://ifmo.ru/etsilence/ontologies

# SHACL

### Install dependencies for SHACL

In [34]:
!pip install SPARQLWrapper
!apt-get install netbase
!pip install pyshacl



Reading package lists... Done
Building dependency tree       
Reading state information... Done
netbase is already the newest version (5.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [52]:
shapes_file = '''
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix sh:    <http://www.w3.org/ns/shacl#> .
@prefix xsd:   <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs:  <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl:   <http://www.w3.org/2002/07/owl#> .
@prefix :   <https://ifmo.ru/etsilence/ontologies/2021/4/network-ontology#> .


:FrequencyRangeConstraints
    a sh:NodeShape ;
    sh:targetClass :FrequencyRange ; # Applies to all persons
    sh:property [              # _:b0
        sh:path :frequencyRangeLowerBound ;       # constrains the values of ex:ssn
        sh:lessThanOrEquals :frequencyRangeHigherBound ;
        sh:maxCount 1
    ] ;
    sh:property [              # _:b0
        sh:path :frequencyRangeHigherBound ;       # constrains the values of ex:ssn
        sh:maxCount 1
    ] ;
.

:NetworkDeviceConstraints
    a sh:NodeShape ;
    sh:targetClass :NetworkDevice ; # Applies to all persons
    sh:property [              # _:b0
        sh:path :deviceMinTemperature ;       # constrains the values of ex:ssn
        sh:lessThanOrEquals :deviceMaxTemperature ;
        sh:maxCount 1
    ] ;
    sh:property [              # _:b0
        sh:path :deviceMaxTemperature ;       # constrains the values of ex:ssn
        sh:maxCount 1
    ] ;
 .
 ###### add SHACL vocabulary ###### 

'''

shapes_file_format = 'turtle'



In [53]:
from pyshacl import validate

conforms, v_graph, v_text = validate(g,
                                     shacl_graph=shapes_file,
                                     target_graph_format='rdfxml',
                                     shacl_graph_format=shapes_file_format,
                                     inference='rdfs')
print(conforms)

Constraint Violation in LessThanOrEqualsConstraintComponent (http://www.w3.org/ns/shacl#LessThanOrEqualsConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:lessThanOrEquals :frequencyRangeHigherBound ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:path :frequencyRangeLowerBound ]
	Focus Node: :InvalidFrequencyRange
	Value Node: Literal("11.0", datatype=xsd:double)
	Result Path: :frequencyRangeLowerBound
	Message: Value of :InvalidFrequencyRange->:frequencyRangeHigherBound < Literal("11.0", datatype=xsd:double)



False
